# Hands-on Modul 2: Eksplorasi Tokenisasi & Embedding

Selamat datang di *notebook* praktis untuk Modul 2!

**Tujuan:** Di *notebook* ini, kita akan menjembatani teori dari **Subtopik 2.3** dengan kode praktis menggunakan *library* `transformers` dari Hugging Face.

Kita akan melakukan dua hal:
1.  **Bagian 1: Tokenisasi (Diagram 2.3)**
    * Mengubah teks mentah $\rightarrow$ Token Subword $\rightarrow$ Token ID.
2.  **Bagian 2: Embedding (Diagram 2.4)**
    * Mengubah Token ID $\rightarrow$ Vektor Embedding (Token Embedding + Positional Encoding).

In [1]:
# SEL PERSIAPAN: Instalasi Library
# Jalankan sel ini terlebih dahulu untuk menginstal library yang kita butuhkan.
# 'transformers' adalah library Hugging Face untuk model dan tokenizer.
# 'torch' (PyTorch) adalah library deep learning yang digunakan transformers di belakang layar.

!pip install transformers torch

## Bagian 1: Tokenisasi (Mereplikasi Diagram 2.3)

Kita akan mengambil sebuah kalimat mentah dan melihat bagaimana *tokenizer* memprosesnya menjadi *Token IDs*.

**Eksperimen:**
Coba ganti nilai `model_name` dan `text_to_tokenize` di bawah ini untuk melihat perbedaannya.
* Coba ganti `model_name` dengan `"gpt2"` atau `"google-bert/bert-base-multilingual-cased"`.
* Coba ganti `text` dengan kalimat yang lebih kompleks atau mengandung kata-kata yang tidak umum.

In [2]:
# 1. Impor kelas yang kita butuhkan
from transformers import AutoTokenizer

# --- KONFIGURASI ---
# Ganti nilai variabel ini untuk bereksperimen:
model_name = "bert-base-uncased"
text_to_tokenize = "Arsitektur Keren"
# --------------------

# 2. Memuat tokenizer
print(f"Memuat tokenizer untuk model: {model_name}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat tokenizer: {e}")
    print("Pastikan nama model sudah benar dan Anda terkoneksi ke internet.")

# 3. Menjalankan proses tokenisasi
# Ini adalah implementasi dari alur di Diagram 2.3
output_tokenizer = tokenizer(text_to_tokenize)

# 4. Menampilkan hasil
print(f"\n--- Hasil Tokenisasi untuk: '{text_to_tokenize}' ---")

# .tokenize() adalah helper untuk melihat token string-nya
subwords = tokenizer.tokenize(text_to_tokenize)
print(f"Token Subword : {subwords}")

# output_tokenizer['input_ids'] adalah hasil akhirnya (Token ID)
input_ids = output_tokenizer['input_ids']
print(f"Token ID (Input IDs) : {input_ids}")

print(f"\nJumlah Subword: {len(subwords)}")
print(f"Jumlah Input ID: {len(input_ids)}")

Memuat tokenizer untuk model: bert-base-uncased...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer berhasil dimuat.

--- Hasil Tokenisasi untuk: 'Arsitektur Keren' ---
Token Subword : ['ars', '##ite', '##kt', '##ur', 'ke', '##ren']
Token ID (Input IDs) : [101, 29393, 4221, 25509, 3126, 17710, 7389, 102]

Jumlah Subword: 6
Jumlah Input ID: 8


### 💡 Analisis Hasil Tokenisasi

Perhatikan output di atas (saat menggunakan `bert-base-uncased`):

* **Token Subword:** `['ars', '##ite', '##kt', '##ur', 'ke', '##ren']`
    Ini adalah implementasi **Subword Tokenization (2.3.1)**. Model ini tidak tahu kata "Arsitektur", jadi ia memecahnya menjadi token yang dikenal: `"ars"`, `"##ite"`, `"##kt"`, `"##ur"` (tanda `##` berarti "lanjutan dari kata sebelumnya").

* **Token ID:** `[101, 29393, 4221, 25509, 3126, 17710, 7389, 102]`
    Perhatikan jumlah ID ada 8, padahal subword-nya ada 6. Ini karena **Special Tokens (Tabel 2.4)** ditambahkan secara otomatis:
    * `101` adalah ID untuk `[CLS]` (token di awal).
    * `102` adalah ID untuk `[SEP]` (token di akhir).

---

## Bagian 2: Embedding (Mereplikasi Diagram 2.4)

Sekarang, kita akan mengambil `input_ids` dari Bagian 1 dan melewatkannya ke model untuk mendapatkan *Embedding* akhirnya.

Kita akan membuktikan bahwa outputnya adalah Tensor (matriks) dengan *shape* (dimensi):
`[Batch Size, Sequence Length, Embedding Dimension]`

In [3]:
# 1. Impor library tambahan
from transformers import AutoModel
import torch

# 2. Memuat model penuh
# Kita menggunakan 'model_name' yang sama dari sel sebelumnya
print(f"Memuat model penuh: {model_name}...")
model = AutoModel.from_pretrained(model_name)
print("Model berhasil dimuat.")

# 3. Menyiapkan Input IDs
# Kita gunakan variabel 'input_ids' dari sel sebelumnya
# Model neural (PyTorch/TensorFlow) membutuhkan input sebagai 'Tensor'
# .unsqueeze(0) menambahkan dimensi 'Batch Size' (kita proses 1 kalimat)
input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)
print(f"\nInput IDs (Tensor): {input_ids_tensor}")

# 4. Mendapatkan Embeddings (Implementasi Diagram 2.4)
# Kita memanggil 'model.embeddings' secara EKSPLISIT
# untuk menjalankan HANYA lapisan pertama (Token Emb + Positional Emb)
print("Menjalankan proses Embedding (Diagram 2.4)...")
with torch.no_grad(): # (Mode 'no_grad' = nonaktifkan kalkulasi gradien)
    embedding_output = model.embeddings(input_ids=input_ids_tensor)

# 5. Menampilkan hasil
print(f"\n--- Hasil Embedding ---")
print(f"Dimensi (Shape) Vektor: {embedding_output.shape}")

Memuat model penuh: bert-base-uncased...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model berhasil dimuat.

Input IDs (Tensor): tensor([[  101, 29393,  4221, 25509,  3126, 17710,  7389,   102]])
Menjalankan proses Embedding (Diagram 2.4)...

--- Hasil Embedding ---
Dimensi (Shape) Vektor: torch.Size([1, 8, 768])


### 💡 Analisis Hasil Embedding

Perhatikan output `Dimensi (Shape) Vektor` di atas. Jika Anda menggunakan `bert-base-uncased`, Anda akan melihat:
`torch.Size([1, 8, 768])`

Mari kita bedah artinya, sesuai **Teori 2.3.2**:

* **`1` (Batch Size):** Kita hanya memproses 1 kalimat.
* **`8` (Sequence Length):** Ini adalah jumlah total token yang kita masukkan (`[CLS]`, `ars`, `##ite`, `##kt`, `##ur`, `ke`, `##ren` `[SEP]`).
* **`768` (Embedding Dimension):** Ini adalah "dimensi tersembunyi" dari model BERT-Base.

Ini adalah bukti nyata dari **Diagram 2.4**. Output `embedding_output` adalah sebuah Tensor (matriks 3D) yang berisi **5 vektor**, di mana **setiap vektor memiliki 768 angka**. Vektor-vektor inilah yang merepresentasikan "Input Vektor Akhir (ke LLM)"—gabungan dari *Token Embedding* (makna) dan *Positional Encoding* (lokasi).

---

### 🎓 Penutup dan Eksperimen Lanjutan

Selamat! Anda telah berhasil melacak alur pra-proses LLM dari teks mentah hingga menjadi vektor yang siap diolah oleh *Multi-Head Attention*.

**Tantangan (Eksperimen):**
* Kembali ke **Sel 4**.
* Ubah `model_name` menjadi `"gpt2"`.
* Jalankan ulang **Sel 4** dan **Sel 7**.
* Bandingkan hasilnya. Apakah *tokenizer* GPT-2 memecah "Arsitektur" secara berbeda? (Petunjuk: lihat token `Ġ`). Berapa dimensi *embedding* (angka 768) dari `gpt2`?